# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [36]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import goog_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv('output_data\cities.csv')

cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,safaga,26.7292,33.9365,27.97,40,1,6.14,EG,1632196444
1,hay river,60.8156,-115.7999,7.99,87,20,2.06,CA,1632196181
2,east london,-33.0153,27.9116,13.52,50,0,7.72,ZA,1632196188
3,vila,42.0304,-8.1588,9.49,93,1,2.14,PT,1632196445
4,upernavik,72.7868,-56.1549,-5.17,81,68,2.06,GL,1632196445


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key= goog_key)

# Plot Heatmap
locations = cities_df[['Lat', 'Lng']]
humidity = cities_df['Humidity'].astype(float)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [50]:
perfect_df = cities_df.loc[(cities_df['Max Temp'] >= 21) & (cities_df['Max Temp'] <= 27) & (cities_df['Wind Speed'] <= 10) 
                          & (cities_df['Cloudiness'] ==0 )]
perfect_df.reset_index(drop = True)
perfect_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,guerrero negro,27.9769,-114.0611,21.73,75,0,5.22,MX,1632196298
14,aksu,41.1231,80.2644,26.27,19,0,1.13,CN,1632196449
16,pacific grove,36.6177,-121.9166,25.11,72,0,0.73,US,1632196450
32,sakakah,29.9697,40.2064,24.76,26,0,2.72,SA,1632196221
46,damietta,31.4165,31.8133,26.94,76,0,3.30,EG,1632196462


# Returning Hotels in Cities with Ideal Weather Conditions

In [34]:
hotel_df = perfect_df.loc[:,["City","Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Rating"] = ""

In [48]:
import json
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword" : "hotel",
    "key": goog_key
}

missing_count = 0
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    city = row['City']

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    print(f"Retrieving results for {city}.")
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
        #add responses to dataframe
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][1]["name"]
        hotel_df.loc[index, "Hotel Rating"] = name_address["results"][1]["rating"]
       
    except (KeyError, IndexError):
        missing_count += 1
        print(f'No rating found for {city}\n')
        hotel_df.loc[index, "Hotel Rating"] = "No rating found for {city}"
        
print(f'Number of Hotels missing rating: {missing_count}')

Retrieving results for guerrero negro.

Retrieving results for aksu.

Retrieving results for pacific grove.

Retrieving results for sakakah.

Retrieving results for damietta.

Retrieving results for pishin.

Retrieving results for gat.

Retrieving results for deming.

Retrieving results for linqing.

Retrieving results for zabol.

No rating found for zabol

Retrieving results for hengshui.

Retrieving results for neyshabur.

Retrieving results for nouadhibou.

Retrieving results for komsomolskiy.

Retrieving results for dale.

Retrieving results for karpathos.

Number of Hotels missing rating: 1


In [44]:
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name,Hotel Rating
10,guerrero negro,MX,27.9769,-114.0611,The Halfway Inn,4.2
14,aksu,CN,41.1231,80.2644,Tianyuan International Hotel,5
16,pacific grove,US,36.6177,-121.9166,The Monterey Hotel,3.9
32,sakakah,SA,29.9697,40.2064,Raoum Inn Hotel,3.9
46,damietta,EG,31.4165,31.8133,Casablanca Hotel,3.5


In [40]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [45]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))